# Day 20

## Part One

<p>With the scanners fully deployed, you turn their attention to mapping the floor of the ocean trench.</p>
<p>When you get back the image from the scanners, it seems to just be random noise. Perhaps you can combine an image enhancement algorithm and the input image (your puzzle input) to clean it up a little.</p>
<p>For example:</p>
<pre><code>..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..##
#..######.###...####..#..#####..##..#.#####...##.#.#..#.##..#.#......#.###
.######.###.####...#.##.##..#..#..#####.....#.#....###..#.##......#.....#.
.#..#..##..#...##.######.####.####.#.#...#.......#..#.#.#...####.##.#.....
.#..#...##.#.##..#...##.#.##..###.#......#.#.......#.#.#.####.###.##...#..
...####.#..#..#.##.#....##..#.####....##...##..#...#......#.#.......#.....
..##..####..#...#.#.#...##..#.#..###..#####........#..####......#..#

#..#.
#....
##..#
..#..
..###
</code></pre>
<p>The first section is the <em>image enhancement algorithm</em>. It is normally given on a single line, but it has been wrapped to multiple lines in this example for legibility. The second section is the <em>input image</em>, a two-dimensional grid of <em>light pixels</em> (<code>#</code>) and <em>dark pixels</em> (<code>.</code>).</p>
<p>The image enhancement algorithm describes how to enhance an image by <em>simultaneously</em> converting all pixels in the input image into an output image. Each pixel of the output image is determined by looking at a 3x3 square of pixels centered on the corresponding input image pixel. So, to determine the value of the pixel at (5,10) in the output image, nine pixels from the input image need to be considered: (4,9), (4,10), (4,11), (5,9), (5,10), (5,11), (6,9), (6,10), and (6,11). These nine input pixels are combined into a single binary number that is used as an index in the <em>image enhancement algorithm</em> string.</p>
<p>For example, to determine the output pixel that corresponds to the very middle pixel of the input image, the nine pixels marked by <code>[...]</code> would need to be considered:</p>
<pre><code># . . # .
#[. . .].
#[# . .]#
.[. # .].
. . # # #
</code></pre>
<p>Starting from the top-left and reading across each row, these pixels are <code>...</code>, then <code>#..</code>, then <code>.#.</code>; combining these forms <code>...#...#.</code>. By turning dark pixels (<code>.</code>) into <code>0</code> and light pixels (<code>#</code>) into <code>1</code>, the binary number <code>000100010</code> can be formed, which is <code>34</code> in decimal.</p>
<p>The image enhancement algorithm string is exactly 512 characters long, enough to match every possible 9-bit binary number. The first few characters of the string (numbered starting from zero) are as follows:</p>
<pre><code>0         10        20        30  <em>34</em>    40        50        60        70
|         |         |         |   <em>|</em>     |         |         |         |
..#.#..#####.#.#.#.###.##.....###.<em>#</em>#.#..###.####..#####..#....#..#..##..##
</code></pre>
<p>In the middle of this first group of characters, the character at index 34 can be found: <code>#</code>. So, the output pixel in the center of the output image should be <code>#</code>, a <em>light pixel</em>.</p>
<p>This process can then be repeated to calculate every pixel of the output image.</p>
<p>Through advances in imaging technology, the images being operated on here are <em>infinite</em> in size. <em>Every</em> pixel of the infinite output image needs to be calculated exactly based on the relevant pixels of the input image. The small input image you have is only a small region of the actual infinite input image; the rest of the input image consists of dark pixels (<code>.</code>). For the purposes of the example, to save on space, only a portion of the infinite-sized input and output images will be shown.</p>
<p>The starting input image, therefore, looks something like this, with more dark pixels (<code>.</code>) extending forever in every direction not shown here:</p>
<pre><code>...............
...............
...............
...............
...............
.....#..#......
.....#.........
.....##..#.....
.......#.......
.......###.....
...............
...............
...............
...............
...............
</code></pre>
<p>By applying the image enhancement algorithm to every pixel simultaneously, the following output image can be obtained:</p>
<pre><code>...............
...............
...............
...............
.....##.##.....
....#..#.#.....
....##.#..#....
....####..#....
.....#..##.....
......##..#....
.......#.#.....
...............
...............
...............
...............
</code></pre>
<p>Through further advances in imaging technology, the above output image can also be used as an input image! This allows it to be enhanced <em>a second time</em>:</p>
<pre><code>...............
...............
...............
..........#....
....#..#.#.....
...#.#...###...
...#...##.#....
...#.....#.#...
....#.#####....
.....#.#####...
......##.##....
.......###.....
...............
...............
...............
</code></pre>
<p>Truly incredible - now the small details are really starting to come through. After enhancing the original input image twice, <code><em>35</em></code> pixels are lit.</p>
<p>Start with the original input image and apply the image enhancement algorithm twice, being careful to account for the infinite size of the images. <em>How many pixels are lit in the resulting image?</em></p>

---

In [1]:
# Initialise
with open('Day20.in') as f:
    algorithm, image = f.read().split('\n\n')
    image = image.split('\n')
    
    # Convert into pixel co-ordinates
    pixels = set()
    for i, row in enumerate(image):
        for j, val in enumerate(row):
            if val == '#':
                pixels.add((i,j))

**Note:** Since `algorithm[0]` implies all pixels surrounded by off pixels will turn on (which accounts for most pixels on the infinite grid). Similarly `algorithm[2**10-1]` implies all pixels surrounded by on pixels will turn off (which will account for most pixels after the first step). To deal with this we'll track on/off pixels alternately.

In [2]:
def enhance(px, pxs, tracking='#', algorithm=algorithm):
    """
    Given a pixel coordinate px, a set of pixels pxs, which type of 
    pixels are being tracked (on/off #/.) and the enhancement algorithm
    find the enhancement for that coordinate.
    """
    x, y = px
    b = ''
    
    if tracking == '#':
        for i in range(x-1,x+2):
            for j in range(y-1,y+2):
                b += '1' if (i,j) in pxs else '0'
    else:
        for i in range(x-1,x+2):
            for j in range(y-1,y+2):
                b += '0' if (i,j) in pxs else '1'
    
    return algorithm[int(b, 2)]

def set_range(s):
    """
    Find the maximum x/y range to enhance over
    """
    x = y = 1000
    X = Y = -1000
    
    for px,py in s:
        if px < x:
            x = px
        if px > X:
            X = px
        if py < y:
            y = py
        if py > Y:
            Y = py
    return range(x-1, X+2), range(y-1, Y+2)

def steps(pxs, n=1, tracking='#', xrange=None, yrange=None, algorithm=algorithm):
    """
    Given a set of pixels, performs n steps of enhancement (default is 1).
    
    To save time, xrange and yrange is just a window that increases in each direction
    at the end of each step.
    
    Note that the pixels being tracked alters between on and off with each step.
    """
    if n > 0:
        new = set()
        if xrange is None:
            xrange, yrange = set_range(pxs)
        for x in xrange:
            for y in yrange:
                if enhance((x,y), pxs, tracking) != tracking:
                    new.add((x,y))
        tracking = '#' if tracking == '.' else '.'
        xrange = range(xrange.start - 1, xrange.stop + 1)
        yrange = range(yrange.start - 1, yrange.stop + 1)
        return steps(new, n=n-1, tracking=tracking, xrange=xrange, yrange=yrange)
    else:
        return pxs

# Solution
print(f"After two iterations, {len(steps(pixels, n=2))} pixels are lit.")

After two iterations, 5571 pixels are lit.


---

## Part Two

<p>You still can't quite make out the details in the image. Maybe you just didn't <a href="https://en.wikipedia.org/wiki/Kernel_(image_processing)" target="_blank">enhance</a> it <span title="Yeah, that's definitely the problem.">enough</span>.</p>
<p>If you enhance the starting input image in the above example a total of <em>50</em> times, <code><em>3351</em></code> pixels are lit in the final output image.</p>
<p>Start again with the original input image and apply the image enhancement algorithm 50 times. <em>How many pixels are lit in the resulting image?</em></p>

---

In [3]:
# Solution
print(f"After fifty iterations, {len(steps(pixels, n=50))} pixels are lit.")

After fifty iterations, 17965 pixels are lit.


---